In [ ]:
from os import listdir
from numpy import array
from keras import initializers
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, model_from_json
from keras.utils import to_categorical, Sequence
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import RMSprop, Adamax
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet
import numpy as np

# import the capsule layers stuff
from capsulelayers import *

from keras import backend as K
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
K.tensorflow_backend._get_available_gpus()

In [ ]:
dir_name = 'resources/train/'

# Read a file and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_data(data_dir):
    text = []
    images = []
    # Load all the files and order them
    all_filenames = listdir(data_dir)
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:] == "npz":
            # Load the images already prepared in arrays
            image = np.load(data_dir+filename)
            images.append(image['features'])
        else:
            # Load the boostrap tokens and rap them in a start and end tag
            syntax = '<START> ' + load_doc(data_dir+filename) + ' <END>'
            # Seperate all the words with a single space
            syntax = ' '.join(syntax.split())
            # Add a space after each comma
            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
    images = np.array(images, dtype=float)
    return images, text

#train_features, texts = load_data(dir_name)

In [ ]:
# Initialize the function to create the vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)
# Create the vocabulary 
tokenizer.fit_on_texts([load_doc('resources/bootstrap.vocab')])

# Add one spot for the empty word in the vocabulary 
vocab_size = len(tokenizer.word_index) + 1
# Map the input sentences into the vocabulary indexes
#train_sequences = tokenizer.texts_to_sequences(texts)
# The longest set of boostrap tokens
#max_sequence = max(len(s) for s in train_sequences)
# Specify how many tokens to have in each input sentence
max_length = 48

def preprocess_data(sequences, features):
    X, y, image_data = list(), list(), list()
    for img_no, seq in enumerate(sequences):
        for i in range(1, len(seq)):
            # Add the sentence until the current count(i) and add the current count to the output
            in_seq, out_seq = seq[:i], seq[i]
            # Pad all the input token sentences to max_sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # Turn the output into one-hot encoding
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # Add the corresponding image to the boostrap token file
            image_data.append(features[img_no])
            # Cap the input sentence to 48 tokens and add it
            X.append(in_seq[-48:])
            y.append(out_seq)
    return np.array(X), np.array(y), np.array(image_data)

#X, y, image_data = preprocess_data(train_sequences, train_features)

In [ ]:
# X, y, image_data
# X.shape = (*,48)
# y.shape = (*,18)
# image_data.shape = (*,256,256,3)
# [image_data, X], y

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, dir_name, list_IDs, max_length, tokenizer, batch_size=1, shuffle=True):
        'Initialization'
        self.dir_name = dir_name
        self.list_IDs = list_IDs
        self.batch_size = batch_size
        self.max_length = max_length
        self.shuffle = shuffle
        self.indexes = range(len(list_IDs))
        self.tokenizer = tokenizer
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __load_data_batch(self, data_dir, list_IDs_temp):
        text = []
        images = []
        # Load files in batch
        for filename in (list_IDs_temp):
            image_file = filename + ".npz"
            # Load the images already prepared in arrays
            image = np.load(data_dir+image_file)
            images.append(image['features'])

            text_file = filename + ".gui"
            # Load the boostrap tokens and rap them in a start and end tag
            syntax = '<START> ' + load_doc(data_dir+text_file) + ' <END>'
            # Seperate all the words with a single space
            syntax = ' '.join(syntax.split())
            # Add a space after each comma
            syntax = syntax.replace(',', ' ,')
            text.append(syntax)
        images = np.array(images, dtype=float)
        return images, text

    def __preprocess_data(self, sequences, features):
        X, y, image_data = list(), list(), list()
        for img_no, seq in enumerate(sequences):
            for i in range(1, len(seq)):
                # Add the sentence until the current count(i) and add the current count to the output
                in_seq, out_seq = seq[:i], seq[i]
                # Pad all the input token sentences to max_sequence
                in_seq = pad_sequences([in_seq], maxlen=self.max_length)[0]
                # Turn the output into one-hot encoding
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # Add the corresponding image to the boostrap token file
                image_data.append(features[img_no])
                # Cap the input sentence to 48 tokens and add it
                X.append(in_seq[-48:])
                y.append(out_seq)
        return np.array(X), np.array(y), np.array(image_data)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Generate data
        train_features, texts = self.__load_data_batch(dir_name, list_IDs_temp)
        
        # Map the input sentences into the vocabulary indexes
        train_sequences = tokenizer.texts_to_sequences(texts)

        X, y, image_data = self.__preprocess_data(train_sequences, train_features)

        return [image_data, X], y

In [ ]:
# Parameters
params = {'batch_size': 1,
          'max_length': max_length,
          'shuffle': True,
          'tokenizer': tokenizer}

# Datasets
# Gather list of files, store in partition
all_filenames = listdir(dir_name)
all_filenames.sort()
filenames = []
for filename in (all_filenames):
    if filename[-3:] == "npz":
        filenames.append(filename[:-4])

num_files = len(filenames)
train_size = int(num_files * 0.9)
val_size = num_files - train_size
train_idx = np.random.randint(num_files, size=train_size)
val_idx = np.random.randint(num_files, size=val_size)
train = [filenames[i] for i in train_idx]
val = [filenames[i] for i in val_idx]
partition = {}
partition['train'] = train
partition['validation'] = val

# Generators
training_generator = DataGenerator(dir_name, partition['train'], **params)
validation_generator = DataGenerator(dir_name, partition['validation'], **params)

In [ ]:
# Create a capule network that takes in the input image (256, 256, 3)
# and does the forward pass (and some convolutional layer first)
# probably need to tune layer parameters such as the # of capsules and dimension of input and output
# in this paper: https://arxiv.org/pdf/1712.03480.pdf, they found that adding more conv layers before the
# capsule layer seemed to help ?

# this paper talks about how they adapted capsule networks for food classification: https://sourcediving.com/capsule-networks-for-food-classification-11e024dd8d5d

def capsule_network_model(x):

    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=16, kernel_size=9, strides=2, padding='valid')
    feature_caps = CapsuleLayer(num_capsule=8, dim_capsule=8, routings=3,
                                 name='featurecaps')(primarycaps)
    flattened_feature_caps = Flatten()(feature_caps)
    
    # should probably the repeating part out of this function to make it cleaner
    repeated_features = RepeatVector(max_length)(flattened_feature_caps)
    return repeated_features


In [ ]:
#Create the encoder
visual_input = Input(shape=(256, 256, 3,))
encoded_image = capsule_network_model(visual_input)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 50, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(128, return_sequences=True)(language_model)
language_model = LSTM(128, return_sequences=True)(language_model)

#Create the decoder
decoder = concatenate([encoded_image, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[visual_input, language_input], outputs=decoder)
optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model-capsnet.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate a description for an image
def generate_desc(loaded_model, tokenizer, photo, max_length):
    photo = np.array([photo])
    # seed the generation process
    in_text = '<START> '
    # iterate over the whole length of the sequence
    #print('\nPrediction---->\n\n<START> ', end='')
    for i in range(150):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = loaded_model.predict([photo, sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += word + ' '
        # stop if we predict the end of the sequence
        #print(word + ' ', end='')
        if word == '<END>':
            break
    return in_text

class BleuCallback(Callback):
    max_length = 48
    
    def __init__(self, texts, train_features, tokenizer):
        self.texts = texts
        self.train_features = train_features
        self.tokenizer = tokenizer
        return
        
    # evaluate the skill of the model
    def evaluate_model(self, model, texts, photos, tokenizer, max_length):
    #def evaluate_model(model, texts, photos, tokenizer, max_length):
        actual, predicted = list(), list()
        # step over the whole set
        for i in range(len(texts)):
            yhat = generate_desc(model, tokenizer, photos[i], max_length)
            # store actual and predicted
            #print('\n\nReal---->\n\n' + texts[i])
            actual.append([texts[i].split()])
            predicted.append(yhat.split())
        # calculate BLEU score
        bleu = corpus_bleu(actual, predicted)
        print('\nBleu Score: {}\n'.format(bleu))
        
    def on_epoch_end(self, epoch, logs={}):
        self.evaluate_model(self.model, self.texts, self.train_features, self.tokenizer, max_length)
        return

In [ ]:
#Save the model for every 2nd epoch
filepath="org-weights-epoch-{epoch:04d}--val_loss-{val_loss:.4f}--loss-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=1)
# Tensorboard
board = TensorBoard(log_dir='./logs', histogram_freq=10, batch_size=10, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
# Bleu Score
#bleu = BleuCallback(texts, train_features, tokenizer)

In [ ]:
# Train the model
#callbacks_list = [checkpoint, board, bleu]
#history = model.fit([image_data, X], y, batch_size=10, shuffle=False, validation_split=0.1, callbacks=callbacks_list, verbose=1, epochs=2)

callbacks_list = [checkpoint]
history = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator, 
                    callbacks=callbacks_list,
                    use_multiprocessing=False,
                    workers=1,
                    shuffle=True,
                    verbose=1,
                    epochs=50)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['training loss', 'validation loss'], loc='upper left')
plt.show()